In [1]:
import Pkg
#Pkg.add("MathOptInterface")
#Pkg.add("GLPK")
Pkg.activate(@__DIR__)
Pkg.instantiate()

Activating environment at `C:\Users\mmart\Desktop\TFM\Julia\notebooks\Project.toml`


In [2]:
using JuMP
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

In [3]:
include("../src/read_data.jl")
include("../JuMP/main_JuMP.jl")

H0 = 20

Constants(10, 10, 5, 2, 76, 1, 1)

In [4]:
JJ=[j for j in 1:J]
PP=[p for p in 1:P]
CC=[c for c in 1:C]
TT=[t for t in 0:H-1]
QQ=[q for q in 1:Q]

2-element Array{Int64,1}:
 1
 2

In [5]:
model = Model(with_optimizer(GLPK.Optimizer, msg_lev = 4));  
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer, max_cpu_time=60.0)` becomes `optimizer_with_attributes(Ipopt.Optimizer, "max_cpu_time" => 60.0)`.
│   caller = ip:0x0
└ @ Core :-1


In [ ]:
@variable(model, x[p=1:P, i=1:C, q=1:Q], Bin)
@variable(model, tc[p=1:P, q=1:Q], lower_bound=0, upper_bound=H)
@variable(model, y[t=0:H-1, j=1:J, q=1:Q], Bin)

@variable(model, Start[q=1:Q], lower_bound=0, upper_bound=H)
@variable(model, End[q=1:Q], lower_bound=0, upper_bound=H)

#objective function
@objective(model, Min, sum(End[q] for q in QQ))

In [ ]:
@constraint(model, [i=1:C], sum(sum(x[p,i,q] for q in subset_crane_pos(CTS, p, bj)) for p in subset_pos(PP, tasks_by_position, i)) == 1)
@constraint(model, [p=1:P], sum(sum(x[p,i,q] for q in subset_crane_pos(CTS, p, bj)) for i in subset_pos(CC, tasks_by_position, p)) == 1)

In [ ]:
for i = 1:C
    for p = 1:P
        for q in setdiff(Set(QQ),Set(subset_crane_pos(CTS, p, bj)))
            @constraint(model, x[p,i,q] == 0)
        end
    end
end

In [ ]:
for p = 1:P
    for q in subset_crane_pos(CTS, p, bj)
        @constraint(model, tc[p,q] <= CTS.H*sum(x[p,i,q] for i in subset_pos(CC, tasks_by_position, p)))
        @constraint(model, tc[p,q] >= 2*sum((task_times[p,i]*x[p,i,q]) for i in subset_pos(CC, tasks_by_position, p)))
    end
end

In [ ]:
for q = 1:Q
    for p = 1:P
        @constraint(model, tc[p,q]-2*sum((task_times[p,i]*x[p,i,q]) for i in CC)+CTS.H*(1-sum(x[p,i,q] for i in subset_pos(CC, tasks_by_position, p))) >= Start[q])
    end
end

In [ ]:
for i = 1:C
    for p in subset_pos(PP, tasks_by_position, i)
        if length(prec[p]) > 0
            for q = 1:Q
                for pp in collect(intersect(Set(subset_pos_crane(CTS, q, bj)),Set(prec[p])))
                    @constraint(model, 2*task_times[p,i] - CTS.H*(2-x[p,i,q]-sum(x[pp,ii,q] for ii in subset_pos(CC, tasks_by_position, pp))) <= tc[p,q] - tc[pp,q])
                end
            end
        end
    end
end

In [ ]:
for p = 1:P
    if length(prec[p]) > 0
        for ppp in sort(prec[p], rev=true)[1]
            @constraint(model, sum(tc[p,q] for q in QQ) >= sum(tc[ppp,q] for q in QQ) - sum(task_times[ppp,i]*sum(x[ppp,i,q] for q in QQ) for i in subset_pos(CC, tasks_by_position, ppp)))
            #@constraint(model, sum(tc[p,q] for q in QQ) >= sum(tc[ppp,q] for q in QQ))
        end
    end
end

In [ ]:
@constraint(model, [p=1:P, q=1:Q], tc[p,q] <= End[q])
@constraint(model, [q=1:Q], Start[q] <= End[q])

## Crane Constraints

In [ ]:
#@constraint(model, [t=0:H-1, q=1:Q], sum(y[t,j,q] for j in JJ) == 1)
@constraint(model, [t=0:H-1, j=1:J], sum(y[t,j,q] for q in QQ) <= 1)

In [ ]:
for t = 0:H-1
    for q = 1:Q
        for j in setdiff(Set(JJ),Set(subset_bay(CTS, q)))
            @constraint(model, y[t,j,q] == 0)
        end
    end
end

In [ ]:
@constraint(model, [t=0:H-1, q=2:Q, j=1:q-1], y[t,j,q] == 0)
@constraint(model, [t=0:H-1, q=1:Q-1, j=J-(Q-q)+1:J], y[t,j,q] == 0)

In [ ]:
@constraint(model, [t=0:H-1, q=1:Q, s=1:Q, j=2:J], 1-y[t,j,q] >= sum(y[t,l,s] for l in collect(max(1,j-d):j-1)))
@constraint(model, [t=0:H-1, q=1:Q, s=1:Q, j=1:J-1], 1-y[t,j,q] >= sum(y[t,l,s] for l in collect(j+1:min(j+d,J))))

@constraint(model, [t=0:H-1, q=1:Q-1, j=1:J-1], y[t,j,q] <= sum(y[t,l,q+1] for l in collect(j+1:J)))
@constraint(model, [t=0:H-1, q=2:Q, j=2:J], y[t,j,q] <= sum(y[t,l,q-1] for l in collect(1:j-1)))

In [ ]:
for t = 0:H-1
    for p = 1:P
        for q in subset_crane_pos(CTS, p, bj)
            @constraint(model, tc[p,q] <= y[t,bj[p],q]*CTS.H)
            @constraint(model, y[t,bj[p],q] <= sum(x[p,i,q] for i in subset_pos(CC, tasks_by_position, p)))
        end
    end
end

## Solution

In [ ]:
JuMP.optimize!(model) # Old syntax: status = JuMP.solve(model)

In [ ]:
sol_x = Dict{Int, Array}()
for p=1:P
    for i=1:C
        for q=1:Q
            if JuMP.value.(x)[p,i,q] == 1
                if haskey(sol_x, q) == false
                    sol_x[q] = Array{NamedTuple{(:pos, :cont),Tuple{Int64,Int64}}, 1}()
                end
                push!(sol_x[q], (pos=p, cont=i))
            end                
        end
    end
end

sol_x[1]

In [ ]:
sol_x[2]

In [ ]:
sol_t = JuMP.value.(tc)

In [ ]:
sol_y = Dict{Int, Array}()
for t=0:H-1
    for j=1:J
        for q=1:Q
            if JuMP.value.(y)[t,j,q] == 1
                if haskey(sol_y, q) == false
                    sol_y[q] = Array{NamedTuple{(:time, :bay),Tuple{Int64,Int64}}, 1}()
                end
                push!(sol_y[q], (time=t, bay=j))
            end                
        end
    end
end

sol_y[1]

In [ ]:
sol_y[2]